<a href="https://colab.research.google.com/github/Jason910315/fine-tuning-llm-model/blob/main/fine_tuning_fortune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
# unsloth 是一個用於大型語言模型微調的工具，可以讓模型運行更快，佔用更少內存
!pip install unsloth

# 卸載當前已安装的 unsloth 包（如果已安装），然後從 Github 安裝最新版本。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# bitsandbytes 是一個用於量化和優化模型的工具，幫著減少模型占用的內存
# unsloth_zoo 包含了一些預訓練模型或其他工具。
!pip install bitsandbytes unsloth_zoo

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-t4uk4fko
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-t4uk4fko
  Resolved https://github.com/unslothai/unsloth.git to commit 06c00b06b44742b5c305226aea43c3b59ab4ca7a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2026.1.1-py3-none-any.whl size=387345 sha256=34fcf6db72b6aea1df02bf317ebf1f2427c4a16f70cef2695d1ce2dd75227fa1
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_b8adyj/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.0 MB/s eta 0:00:00
   ━━━

# Load pretrained model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # 模型處理上下文的最大長度
dtype = None           # 資料類型，讓模型選擇最適合的精度
load_in_4bit = True    # 使用 4bits 減少模型精度以節省內存

# 載入預訓練模型，並獲得 tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.1: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

# Test

In [ ]:
# 定義提示風格
prompt_style = """以下是描述任務的指令，以及提供進一步上下文的輸入。
請寫出一個適當完成請求的回答。
在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。

### 指令：
你是一位精通卜卦、星象和運勢預測的算命大師。
請回答以下算命問題

### 問題：
{}

### 回答：
<think>{}"""
question = "1992年閏年四月初九巳时生人，女，想了解健康運勢"

In [ ]:
# 準備推理模型
FastLanguageModel.for_inference(model)
# 把 question、空字串傳入 template 的空格，並將文字轉成模型可以吃的張量輸入
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 讓模型根據問題生成回答，最多 1200 個詞
outputs = model.generate(
    input_ids=inputs.input_ids,  # 輸入數字序列
    attention_mask=inputs.attention_mask,  # 註意力遮罩，讓模型知道哪部分更重要
    max_new_tokens=1200,
    use_cache=True,  # 使用緩存加速生成
)

# 解碼模型輸出為可讀文本
response = tokenizer.batch_decode(outputs)
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任務的指令，以及提供進一步上下文的輸入。
請寫出一個適當完成請求的回答。
在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。

### 指令：
你是一位精通卜卦、星象和運勢預測的算命大師。
請回答以下算命問題

### 問題：
1992年閏年四月初九巳时生人，女，想了解健康運勢

### 回答：
<think>
嗯，我现在需要帮一位1992年4月9日蛇年（巳时）出生的女性分析她的健康运势。首先，我得回顾她的出生年份和时间，看看有什么特殊之处。1992年是蛇年，这意味着她出生在蛇年，可能会带来一些与蛇相关的特质，比如灵活、智慧或者保护欲。

接下来，考虑她的生肖是蛇，而她的问题是关于健康，所以我要看看蛇在命理中的健康方面有什么特点。蛇通常被认为具有力量和生命力，但同时也可能带来一些挑战，比如需要保护自己免受外界的伤害，或者需要注意身体的某些方面。

然后，考虑她的生日是4月9日，巳时（相当于上午7点到9点之间）。巳时通常与智慧、判断力和决策有关，可能意味着她在处理健康问题时需要特别注意细节和判断力，可能需要更多的信息来做出决策。

作为女性，她在健康方面可能会面临一些特定的挑战，比如情绪管理、压力处理或者维护身体的平衡。考虑到她可能会比较注重外表和形象，可能在选择健康方式时会倾向于更注重美容和养生，而忽略一些基础的身体健康维护。

此外，她的运势可能涉及到五行的平衡。蛇是木元素，可能需要注意木火土的平衡，避免过于强调某一方面。她的健康可能需要关注心脏、血液循环或者消化系统，因为蛇通常与这些方面相关联。

她可能需要在健康方面多关注一些具体的方面，比如心脏健康、血液循环或者消化系统的健康。同时，可能需要注意情绪波动，保持心情舒畅，避免长期的压力和焦虑。

最后，我需要综合这些因素，给出她健康运势的建议。建议她关注心脏健康，保持良好的情绪，注意身体的各个方面，必要时寻求专业的医疗建议，保持健康的生活方式，平衡工作和休息，关注营养和运动。
</think>

1992年4月9日出生的女性，她的健康运势可以从她的出生年份和时间以及生肖来分析。

首先，1992年是蛇年，蛇象征着力量、生命力和智慧，但同时也可能带来保护欲和身体方面的挑战。作为女性，她在健康方面可能会面临一些特定的挑战，比如情绪管理和

# Load dataset

In [ ]:
# 定義格式化提示的字串，用於微調的模板(提示詞將模型角色的形容更加精準)
train_prompt_style = """以下是描述任務的指令，以及提供進一步上下文的輸入。
請寫出一個適當完成請求的回答。
在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。

### 指令：
你是一位精通八字算命、 紫微斗數、 風水、易經卦象、塔羅牌占卜、星象、面相手相和運勢預測等方面的算命大师。
請回答以下算命問題

### 問題：
{}

### 回答：
<思考>
{}
</思考>
{}"""


In [ ]:
EOS_TOKEN = tokenizer.eos_token

from datasets import load_dataset
# 以前 200 筆數據作為訓練，並使用預設的語言(default)
dataset = load_dataset("Conard/fortune-telling", 'default', split = "train[0:200]", trust_remote_code=True)
print(dataset.column_names)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'Conard/fortune-telling' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'Conard/fortune-telling' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

all_details.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/207 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


In [ ]:
# 格式化資料集中的每筆記錄
def formatting_prompts_func(examples):
  # 從資料集提取問題、思考過程、以及回答
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]
  texts = []  # 儲存格式化後的文本，用於輸入到微調
  # 將每個問題、思考過程、回答都嵌入提示詞模板，並輸入到 texts
  for input, cot, output in zip(inputs, cots, outputs):
    # 最後為每個紀錄加上結尾
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)
  # 返回所有格式化文本的字典
  return {
      "text": texts,
  }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]  # 印出第一筆記錄來觀察


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任務的指令，以及提供進一步上下文的輸入。\n請寫出一個適當完成請求的回答。\n在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。\n\n### 指令：\n你是一位精通八字算命、 紫微斗數、 風水、易經卦象、塔羅牌占卜、星象、面相手相和運勢預測等方面的算命大师。\n請回答以下算命問題\n\n### 問題：\n新房装修,大门对着电梯好不好?要如何化解?\n\n### 回答：\n<思考>\n好的，用户问的是新房装修时大门对着电梯好不好，以及如何化解。首先，我需要回忆一下风水学中关于大门和电梯的相关知识。电梯在风水中属于动气比较强的地方，因为电梯频繁开合，会带来不稳定的气流，也就是所谓的“煞气”。大门是住宅的纳气口，如果正对电梯，可能会让这些不稳定的气流直接冲进家里，影响居住者的健康和财运。\n\n接下来，我需要确认用户的具体情况。比如，大门和电梯的距离有多远？是否正对还是稍微偏一点？不过用户没有提供这些细节，所以只能给出一般性的建议。化解的方法通常有几种：屏风或玄关、门帘、五帝钱、植物、八卦镜等。需要逐一解释这些方法的原理和使用方式，同时提醒用户要根据实际情况选择，必要时咨询专业风水师。\n\n另外，还要注意语气要亲切，避免使用过于专业的术语，让用户容易理解。同时，要强调这些是传统方法，效果因人而异，保持客观中立。最后，可以建议用户如果情况复杂，最好请专业人士实地查看，这样更稳妥。\n\n</思考>\n根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：\n\n1. 玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带\n\n2. 五行通关法\n门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜\n\n3. 光影化解术\n安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域\n\n4. 现代科技方案\n入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场\n\n需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。<｜end▁of▁sentence｜>'

# Fine tuning

In [ ]:
FastLanguageModel.for_training(model)

# 使用 Parameter-Efficient Fine Tuning，並將模型轉為準備微調的狀態
model = FastLanguageModel.get_peft_model(
    model,     # 已經加載過的預訓練模型
    r = 16,     # LoRA 的的秩，決定可訓練的參數量
    # 指定模型中需要微調的區塊，包括 Attention 的 query、key、 value、 output
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 設定 LoRA 的超參數
    lora_dropout = 0, # 設定參數丟棄綠
    bias = "none",    # 是否添加偏置項
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2026.1.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer  # SFFTaininer，用於監督式微調
from transformers import TrainingArguments  # 用於設定訓練參數
from unsloth import is_bfloat16_supported

# 創建一個 SFFTrainer 物件
trainer = SFTTrainer(
    model=model,   # 傳入要微調的模型
    tokenizer=tokenizer,  # 用於處理文本數據
    train_dataset=dataset,  # 訓練資料集
    dataset_text_field="text",  # 指定訓練資料集中的文本標題 (上面定義過的)
    max_seq_length=max_seq_length,
    dataset_num_proc=2,    # 資料處理的並行數
    packing=False,
    # 定義訓練參數
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 每個設備 (如 GPU) 上的批量大小
        warmup_steps=5,  # 預熱，訓練開始時學習綠開始增加的步數
        max_steps=75,  # 最大訓練步數 (epoches)
        learning_rate=2e-4,  # 學習率
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # 每隔多少步記錄一次日誌
        optim="adamw_8bit",  # 使用的優化器，用於調整模型參數
        weight_decay=0.01,  # 權重衰減，防止過擬合
        lr_scheduler_type="linear",  # 控制學習率的變化方式
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# 開始 fine-tuning
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 1 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.273100
2,2.589100
3,2.227500
4,2.432700
5,1.999500
6,2.067100
7,2.357200
8,1.713200
9,2.127400
10,1.912500


# Test Fine tuning

In [ ]:
# 再次印出問體檢視
print(question)

1992年閏年四月初九巳时生人，女，想了解健康運勢


In [ ]:
# 將模型切換到推理模式，準備回答問題
FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=4000,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任務的指令，以及提供進一步上下文的輸入。
請寫出一個適當完成請求的回答。
在回答之前，請仔細思考問題，並創建一個邏輯連貫的思考過程，以確保回答準確無誤。

### 指令：
你是一位精通卜卦、星象和運勢預測的算命大師。
請回答以下算命問題

### 問題：
1992年閏年四月初九巳时生人，女，想了解健康運勢

### 回答：
<think>
<思考>
好的，用户提供了一个详细的命理查询，涉及到生辰和健康运势。首先，我需要确认用户提供的生辰是否正确，包括年份、月份、日、时，以及生肖和性别。用户给出的生辰是1992年闰年四月初九巳时，生肖是鸡，性别是女性。

接下来，我需要分析用户的需求。用户想了解健康运势，可能她目前面临一些健康问题，或者想预防潜在的问题。作为命理师，我需要结合传统命理中的五行、八卦、流年运势以及生辰中的吉凶星来分析健康方面的建议。

首先，五行分析。1992年的干支是癸卯，癸属水，卯属木。四月初九巳时，巳属火。火木相生，水木相克。用户的生肖是鸡，鸡属金，金生水，水生木。综合来看，五行中有火、木、水的关系，需要平衡这些元素，避免过多的火或木，可能导致健康问题。

八卦分析。1992年是癸卯年，属水木，四月初九巳时，属火。八卦组合为癸卯木水，巳时火，形成癸卯木水火。木克水，火生木，整体可能偏向火旺，容易导致火性疾病或过度消耗。需要用土来调和木，土生火，火生土，形成循环，保持平衡。

流年运势。2023年是癸卯年，和用户的生辰相同，流年中可能会有火旺的影响，建议注意心血管健康，避免过度劳累。2024年是甲辰年，辰属土，土生火，可能需要用土来调和火，建议多吃土类食物，增强免疫力。

健康方面建议：首先，调整饮食，多吃水果蔬菜，适量补充肉类，保持低脂饮食。其次，注意劳逸结合，避免熬夜，保持良好的作息。还可以通过太极拳等气功增强体质。另外，避免过度情绪波动，保持心情平和。

需要注意的是，传统命理建议只是参考，结合现代医学进行综合治疗更为重要。用户如果有具体健康问题，应及时咨询专业医生。同时，建议用户保持积极心态，健康管理需要综合因素，不能单纯依赖命理。

最后，确保回答准确，符合传统命理的逻辑，同时给予用户实际可行的建议，帮助她更好地关注健康。避免使用过于神秘化的语言，保持专业但易于理解的风格。

</思考>


# Save model to GGUF

In [ ]:
# GGUF 是一種高效存取 LLM 模型的格式，支持多种量化方法 (4、8、16bit)，可以顯著減小模型的大小，便於儲存
# 例如在 Ollama 上部署時，量化後的模型在資源受限的設備上運行更快。

# 用於存取用戶資料
from google.colab import userdata

# 從 Colab 設定中獲得 huggingface 的密鑰
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 保存為 8bit 量化格式（Q8_0）
# 這種格式文件小且運行快，適合部署到資源受限的設備
if True: model.save_pretrained_gguf("model", tokenizer,)

# 保存為 16bit 量化格式
# 量化精度更高，但文件較大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 保存為 4bit 量化格式（q4_k_m）
# 文件最小，但精度稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [12:23<00:00, 185.87s/it]


Unsloth: Merge process complete. Saved to `/content/model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['deepseek-r1-distill-llama-8b.F16.gguf']
Unsloth

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: All GGUF conversions completed successfully!
Generated files: ['deepseek-r1-distill-llama-8b.Q8_0.gguf']
Unsloth: No Ollama template mapping found for model 'unsloth/deepseek-r1-distill-llama-8b'. Skipping Ollama Modelfile
Unsloth: example usage for text only LLMs: llama-cli --model deepseek-r1-distill-llama-8b.Q8_0.gguf -p "why is the sky blue?"


# Upload to HuggingFace

In [ ]:
# 用於創建新的 model repository
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上創建一個新的倉庫
create_repo("Jasonyang0315/fortunetelling_model", token=HUGGINGFACE_TOKEN, exist_ok=True)

# 將模型和 tokenizer 上傳
model.push_to_hub_gguf("Jasonyang0315/fortunetelling_model", tokenizer, token=HUGGINGFACE_TOKEN)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [07:11<00:00, 107.83s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [03:58<00:00, 59.60s/it]


Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_x5fhsyqf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
